### input
crsp.csv - marketing data by year     
comp.csv - accounting data by year
### output
final_variables.csv with all features, bankruptcy indicator Y, and fyear, cusip, permno, permco


In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from IPython.core.interactiveshell import InteractiveShell
import time
InteractiveShell.ast_node_interactivity = "all" # display any variable and statement without a print in a cell

In [2]:
def print_null_freq(df):
    """ calculate null value # table for a df
    :param df: dataframe to look into
    :type df: pandas dataframe
    :rtype: stdout
    """
    df_lng = pd.melt(df)
    null_variables = df_lng.value.isnull()
    return pd.crosstab(df_lng.variable, null_variables)

### loading data
- combine two tables using 'Permno' and 'fyear'
- combine comp data bk indicator 'dlrsn==2', 'dlrsn==3' and market data bk indicator 'D==1', get final bk indicator Y
- output: datafram final, combined comp and market data from 1980-2015

In [3]:
market = pd.read_csv('data/crsp.csv')
market = market.drop('Unnamed: 0',1)
comp = pd.read_csv('data/comp.csv')
#combine comp delist reason
comp_delist = comp[(comp.dlrsn == 2) |(comp.dlrsn == 3)]
comp_delist_gvkey = comp_delist['gvkey'].unique()
# For any bk compnay in comp, only assigns the max_year of that company as Y = 1
comp['y_from_comp'] = 0
for gvkey in comp_delist_gvkey:
    temp = comp[comp['gvkey'] == gvkey]
    max_year = temp['fyear'].max()
    comp.ix[(comp['gvkey']==gvkey) & (comp['fyear'] == max_year),'y_from_comp'] = 1
final = comp.merge(market, on=['PERMNO','fyear'], how='inner')
final['Y'] = 0
final.ix[(final['D']==1) | (final['y_from_comp']==1),'Y'] = 1

In [4]:
comp[comp['cusip8'] == '00037010']

,gvkey,datadate,fyear,cusip,act,ap,at,ch,che,dlc,...,MVE,QA,dlrsn,sic,dldte,tic,cusip8,PERMNO,PERMCO,y_from_comp
53,1005,19811031,1981,000370106,17.223,3.437,24.480,NaN,0.412,3.468,...,13.383248,10.640,1,3724,19830131,ABA.2,00037010,61903,11,0
54,1005,19801031,1980,000370106,10.275,2.806,15.504,NaN,0.918,0.042,...,14.447997,6.977,1,3724,19830131,ABA.2,00037010,61903,11,0
55,1005,19791031,1979,000370106,6.241,1.130,9.251,0.263,0.263,1.388,...,2.507499,3.806,1,3724,19830131,ABA.2,00037010,61903,11,0


### Clean data

In [5]:
#handle missing values in final data
del final['ffo']
final = final.dropna(subset=['at'])
final = final.dropna(subset=['seq'])
final = final.dropna(subset=['lt'])
final['ni'] = final['ni'].fillna(0)
final['sale'] = final['sale'].fillna(0)
final['oiadp'] = final['oiadp'].fillna(0)
final['ebit'] = final['ebit'].fillna(0)
final['dp'] = final['dp'].fillna(0)
final['re'] = final['re'].fillna(0)
final['lct'] = final['lct'].fillna(0)
final['ch'] = final['ch'].fillna(0)
final['lt'] = final['lt'].fillna(0)
final['QA'] = final['QA'].fillna(0)
final['act'] = final['act'].fillna(0)
final['wcap'] = final['wcap'].fillna(0)
final['invt'] = final['invt'].fillna(0)
final['ap'] = final['ap'].fillna(0)
final['invch'] = final['invch'].fillna(0)
final['che'] = final['che'].fillna(0)
final['dlc'] = final['dlc'].fillna(0)
final['dltt'] = final['dltt'].fillna(0)
final['mib'] = final['mib'].fillna(0)
final['BE'] = final['BE'].fillna(0)
# remove the rows where lct is bigger than lt
final['compare_lt_lct'] = final.apply(lambda x: 1 if x['lct']>x['lt'] else 0, axis = 1)
final.drop(final[final.compare_lt_lct==1].index,inplace = True)
#create market value of total asset, mta
final['mta'] = final['mket']+final['lt']+final['mib']

# all the functions to calculate certain variables
def pretty_division(x,y):
    if y == 0:
        return 0
    return x/y
def NIAT(df):
    return pretty_division(df['ni'],df['at'])
def NISALE(df):
    return pretty_division(df['ni'],df['sale'])
def OIADPAT(df):
    return pretty_division(df['oiadp'],df['at'])
def OIADPSALE(df):
    return pretty_division(df['oiadp'],df['sale'])
def EBITAT(df):
    return pretty_division(df['ebit'],df['at'])
def EBITDPAT(df):
    return pretty_division((df['ebit']+df['dp']),df['at'])
def EBITSALE(df):
    return pretty_division(df['ebit'],df['sale'])
def SEQAT(df):
    return pretty_division(df['seq'],df['at'])
def REAT(df):
    return pretty_division(df['re'],df['at'])
def LCTAT(df):
    return pretty_division(df['lct'],df['at'])
def LCTCHAT(df):
    return pretty_division((df['lct']-df['ch']),df['at'])
def LTAT(df):
    return pretty_division(df['lt'],df['at'])
def CHAT(df):
    return pretty_division(df['ch'],df['at'])
def CHLCT(df):
    return pretty_division(df['ch'],df['lct'])
def QALCT(df):
    return pretty_division(df['QA'],df['lct'])
def ACTLCT(df):
    return pretty_division(df['act'],df['lct'])
def WCAPAT(df):
    return pretty_division(df['wcap'],df['at'])
def LCTLT(df):
    return pretty_division(df['lct'],df['lt'])
def INVTSALE(df):
    return pretty_division(df['invt'],df['sale'])
def SALEAT(df):
    return pretty_division(df['sale'],df['at'])
def APSALE(df):
    return pretty_division(df['ap'],df['sale'])
def INVCHINVT(df):
    return pretty_division(df['invch'],df['invt'])
def CASHAT(df):
    return pretty_division(df['che'],df['at'])
def FFOLT(df):
    return pretty_division(df['ffo'],df['lt'])
def LCTSALE(df):
    return pretty_division(df['lct'],df['sale'])
def RELCT(df):
    return pretty_division(df['re'],df['lct'])
def FAT(df):
    return pretty_division((df['dlc']+0.5*df['dltt']),df['at'])
def NIMTA(df):
    return pretty_division(df['ni'],df['mta'])
def LTMTA(df):
    return pretty_division(df['lt'],df['mta'])
def CASHMTA(df):
    return pretty_division(df['che'],df['mta'])
def RSIZE(df):
    return np.log(pretty_division((1000*df['mket']),df['totval']))
def EXCESS_RETURN(df):
    return (np.log(1+df['RET']) - np.log(1+df['vwretd']))
def MBE(df):
    return pretty_division(df['mket'],(df['BE']+0.1*(df['mket']-df['BE'])))

### Calculating all variables

In [29]:
#define a function get_variables() to get a df with all clean features and bankruptcy indicator Y
def get_variables(df_raw):
    df_variables = pd.DataFrame()
    df_variables['Y'] = df_raw['Y']
    df_variables['gvkey'] = df_raw['gvkey']
    df_variables['datadate'] = df_raw['datadate']
    df_variables['fyear'] = df_raw['fyear']
    df_variables['cusip'] = df_raw['cusip']
    df_variables['PERMNO'] = df_raw['PERMNO']
    df_variables['PERMCO'] = df_raw['PERMCO']
    #accounting
    df_variables['NIAT'] = df_raw.apply(NIAT,axis=1)
    df_variables['NISALE'] = df_raw.apply(NISALE,axis=1)
    df_variables['OIADPAT'] = df_raw.apply(OIADPAT,axis=1)
    df_variables['OIADPSALE'] = df_raw.apply(OIADPSALE,axis=1)
    df_variables['EBITAT'] = df_raw.apply(EBITAT,axis=1)
    df_variables['EBITDPAT'] = df_raw.apply(EBITDPAT,axis=1)
    df_variables['EBITSALE'] = df_raw.apply(EBITSALE,axis=1)
    df_variables['SEQAT'] = df_raw.apply(SEQAT,axis=1)
    df_variables['REAT'] = df_raw.apply(REAT,axis=1)
    df_variables['LCTAT'] = df_raw.apply(LCTAT,axis=1)
    df_variables['LCTCHAT'] = df_raw.apply(LCTCHAT,axis=1)
    df_variables['LTAT'] = df_raw.apply(LTAT,axis=1)
    df_variables['LOGSALE'] = np.log(abs(df_raw['sale'])) # if sale is 0, logsale will be 0 here
    df_variables['CHAT'] = df_raw.apply(CHAT,axis=1)
    df_variables['CHLCT'] = df_raw.apply(CHLCT,axis=1)
    df_variables['QALCT'] = df_raw.apply(QALCT,axis=1)
    df_variables['ACTLCT'] = df_raw.apply(ACTLCT,axis=1)
    df_variables['WCAPAT'] = df_raw.apply(WCAPAT,axis=1)
    df_variables['LCTLT'] = df_raw.apply(LCTLT,axis=1)
    df_variables['INVTSALE'] = df_raw.apply(INVTSALE,axis=1)
    df_variables['SALEAT'] = df_raw.apply(SALEAT,axis=1)
    df_variables['APSALE'] = df_raw.apply(APSALE,axis=1)
    df_variables['LOGSALE'] = np.log(abs(df_raw['sale']))
    df_variables['LOGAT'] = np.log(df_raw['at']) # if total asset is 0, logat will be 0 here
    df_variables['INVCHINVT'] = df_raw.apply(INVCHINVT,axis=1)
    df_variables['CASHAT'] = df_raw.apply(CASHAT,axis=1)
    #df_variables['FFOLT'] = df_raw.apply(FFOLT,axis=1)
    df_variables['LCTSALE'] = df_raw.apply(LCTSALE,axis=1)
    df_variables['RELCT'] = df_raw.apply(RELCT,axis=1)
    df_variables['FAT'] = df_raw.apply(FAT,axis=1)


    # market
    df_variables['SIGMA'] = df_raw['sigma']
    df_variables['NIMTA'] = df_raw.apply(NIMTA,axis=1)
    df_variables['LTMTA'] = df_raw.apply(LTMTA,axis=1)
    df_variables['CASHMTA'] = df_raw.apply(CASHMTA,axis=1)
    df_variables['PRICE'] = df_raw['PRC']
    df_variables['RSIZE'] = df_raw.apply(RSIZE,axis=1)
    df_variables['EXCESS_RETURN'] = df_raw.apply(EXCESS_RETURN,axis=1)
    df_variables['MBE'] = df_raw.apply(MBE,axis=1)
    return df_variables

start_time = time.time()
final_variable = get_variables(final)
print("creating features: %s minutes ---" % round(((time.time() - start_time)/60),2))
final_variable.to_csv('data/final_variables.csv')
#%time

creating features: 2.03 minutes ---


### clean data
### after calculation variables
- 2 infinite variables due to logrithm of 0, logsale and logat
- create table one_year, with Y and x one year before

In [3]:
final_variable = pd.read_csv('data/final_variables.csv')
final_variable = final_variable.drop('Unnamed: 0',1)
final_variable = final_variable.replace([np.inf,-np.inf],0)
final_variable.shape

(182300, 43)

In [15]:
final_variable['Y'].sum()

1162

In [4]:
# merge 'Y' with the data one year before
dat_tmp = final_variable.copy()
dat_tmp['fyear'] = dat_tmp['fyear'] +1
dat_tmp = dat_tmp.drop('Y',axis =1)
Ys = final_variable[['fyear','gvkey','Y']]
one_year = pd.merge(dat_tmp,Ys,how = 'inner',on=['fyear','gvkey'])

## one year model
input files:    
final_variables.csv    
output:     
logistic regression using L1 penalty, one-year time window     
report in sample(1980-2007),out-of sample(2008-2015) roc_auc, accuracy_ratio = (roc_auc-0.5)*2 and deciles of bk  

In [17]:
one_year['Y'].sum()

1016

### train, test split

In [5]:
#seperate training and testing set:from tian's paper, training set is variables before 2003
#testing set is variables after 2003
train = one_year[one_year['fyear']<2008]
test = one_year[one_year['fyear']>=2008]
drop_list = ['gvkey','datadate','fyear','cusip','PERMNO','PERMCO']
train = train.drop(drop_list,axis = 1)
test = test.drop(drop_list,axis = 1)
x_train = train.ix[:,0:-1]
y_train = train.ix[:,-1]
x_test = test.ix[:,0:-1]
y_test = test.ix[:,-1]

In [6]:
train.shape

(131708, 37)

In [7]:
# functions to report result
def get_prob_auc(clf,x,y):
    probas_= clf.predict_proba(x)
    probas_=probas_[:,1]
    fpr,tpr,thresholds = roc_curve(y,probas_)
    roc_auc = roc_auc_score(y,probas_)
    accuracy_ratio = (roc_auc-0.5)*2
    return probas_,accuracy_ratio
def tencile_table(test,p):
    tenc_dat = pd.DataFrame({'y_true':test,'probability':p})
    tenc_dat.sort('probability',axis = 0,ascending=False, inplace = True)
    tenc_dat.index = range(0,len(tenc_dat))
    y = tenc_dat['y_true']
    point = float(len(tenc_dat))/10
    point = int(round(point))
    tenc = []
    for i in range(0,10):
        tenc.append(y[(i*point):((i+1)*point)])
    tenc[9]=tenc[9].append(y[10*point:])
    total = sum(y)
    num_of_bkr = []
    for j in range(0,10):
        num_of_bkr.append(sum(tenc[j]))
    tencile_bkr = np.array(num_of_bkr)
    rate = tencile_bkr.astype(float)/total
    tencile_result=pd.DataFrame({'Group':range(1,11),'Rate':rate})
    return tencile_result

### Use LogisticRegressionCV to find optimal C from logspace(-4,4), CV = 3, metric = AUC, C is the inverse of $\lambda$
- it will take about 160 mins to run, get optimal C = 0.359
- choice of Cs, array([  1.00000000e-04,   7.74263683e-04,   5.99484250e-03,4.64158883e-02,   3.59381366e-01,   2.78255940e+00,2.15443469e+01,   1.66810054e+02,   1.29154967e+03,1.00000000e+04])

In [ ]:
start_time = time.time()

from sklearn.linear_model import LogisticRegressionCV
lgr = LogisticRegressionCV(Cs = np.logspace(-4,4,10),cv=3,
                           penalty = 'l1',
                          scoring='roc_auc',
                          solver= 'liblinear',n_jobs = 2)
fitmodel = lgr.fit(x_train,y_train)
print("grid search to find optimal C: %s minutes ---" % round(((time.time() - start_time)/60),2))
#%time

In [ ]:
print(lgr.C_)
print(lgr.Cs_)
print(lgr.Coef_)

In [8]:
# use logistic regression C = 0.359
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression(C = 0.359,penalty='l1')
lg.fit(x_train,y_train)

LogisticRegression(C=0.359, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [9]:
feature_name = x_train.columns
model_coef = lg.coef_[0]
selected_features = {}
for idx,name in enumerate(feature_name):
    if model_coef[idx] != 0:
        selected_features[name] = model_coef[idx]
#print (selected_features)
print (selected_features.keys())

dict_keys(['PRICE', 'LCTCHAT', 'LTMTA', 'ACTLCT', 'INVCHINVT', 'SALEAT', 'QALCT', 'APSALE', 'LOGAT', 'RELCT', 'REAT', 'OIADPSALE', 'FAT', 'LOGSALE', 'CASHMTA', 'RSIZE', 'CASHAT', 'LCTLT', 'NIMTA', 'CHLCT', 'SIGMA', 'NISALE', 'EXCESS_RETURN', 'EBITDPAT', 'LCTSALE', 'WCAPAT', 'OIADPAT', 'LTAT', 'INVTSALE', 'NIAT', 'MBE'])


In [10]:
# in-sample logistic regression result 
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
in_prob_,in_accuracy_ratio = get_prob_auc(lg, x_train,y_train)
print (tencile_table(y_train,in_prob_))
print ('in-sample logistic regression accuracy ratio is %f, auc is %f' %(in_accuracy_ratio, in_accuracy_ratio/2+0.5))

   Group      Rate
0      1  0.444688
1      2  0.175246
2      3  0.105148
3      4  0.090909
4      5  0.049288
5      6  0.058050
6      7  0.035049
7      8  0.015334
8      9  0.009858
9     10  0.016429
in-sample logistic regression accuracy ratio is 0.578781, auc is 0.789390


In [11]:
# out-sample logistic regression result 
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
out_prob_,out_accuracy_ratio = get_prob_auc(lg, x_test,y_test)
print (tencile_table(y_test,out_prob_))
print ('out-sample logistic regression accuracy ratio is %f, auc is %f' %(out_accuracy_ratio, out_accuracy_ratio/2+0.5))

   Group      Rate
0      1  0.388350
1      2  0.223301
2      3  0.184466
3      4  0.077670
4      5  0.038835
5      6  0.009709
6      7  0.009709
7      8  0.019417
8      9  0.019417
9     10  0.029126
out-sample logistic regression accuracy ratio is 0.578918, auc is 0.789459


In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
clf_rfr = RandomForestClassifier()
clf_rfr.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [13]:
# in-sample rfr result 
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
in_prob_,in_accuracy_ratio = get_prob_auc(clf_rfr, x_train,y_train)
print (tencile_table(y_train,in_prob_))
print ('in-sample random forest accuracy ratio is %f, auc is %f' %(in_accuracy_ratio, in_accuracy_ratio/2+0.5))

   Group  Rate
0      1     1
1      2     0
2      3     0
3      4     0
4      5     0
5      6     0
6      7     0
7      8     0
8      9     0
9     10     0
in-sample random forest accuracy ratio is 0.998788, auc is 0.999394


In [14]:
# out-sample rfr result 
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
out_prob_,out_accuracy_ratio = get_prob_auc(clf_rfr, x_test,y_test)
print (tencile_table(y_test,out_prob_))
print ('out-sample random forest accuracy ratio is %f, auc is %f' %(out_accuracy_ratio, out_accuracy_ratio/2+0.5))

   Group      Rate
0      1  0.330097
1      2  0.116505
2      3  0.077670
3      4  0.097087
4      5  0.097087
5      6  0.029126
6      7  0.029126
7      8  0.077670
8      9  0.058252
9     10  0.087379
out-sample random forest accuracy ratio is 0.241300, auc is 0.620650
